# Import libararies and load data

In [1]:
### ALL NECESSARY LIBRARIES ###
import pandas as pd
import warnings
warnings.simplefilter(action='ignore')

# for saving variables
import pickle
import os
import tiktoken

# needed for gpt
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt


%run "./utilityFunctions.ipynb"

In [2]:
# import data
transcripts = pd.read_csv('ellieAndParticipantTranscripts.csv')

with open('e_daic_phq8_scores_lang.pkl', 'rb') as f:
    phq8_scores = pickle.load(f)


phq8_scores.rename(columns={'Participant_ID': 'ParticipantID'}, inplace=True)
# phq8_scores.head()

merged_data = pd.merge(transcripts, phq8_scores, on='ParticipantID')
merged_data.head()
# merged_data.to_csv('PHQ8_Transcript_Combined.csv', index=False)

,ParticipantID,Conversation,Gender,PHQ_Binary,PHQ_Score,PCL-C (PTSD),PTSD Severity,PHQ_8NoInterest,PHQ_8Depressed,PHQ_8Sleep,PHQ_8Tired,PHQ_8Appetite,PHQ_8Failure,PHQ_8Concentrating,PHQ_8Moving,PHQ_8Total,PHQ8_Binary,PHQ8_Score
0,300,how are you doing today. good. where are you f...,male,0,2,0,25.0,"Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...","Several days over the last 2 weeks, I was both...","Over the last 2 weeks, I was not at all bother...","Several days over the last 2 weeks, I was both...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...",2,This patient has not been diagnosed with depre...,Total PHQ8 score is 2.
1,301,how are you doing today. i'm doing good thank ...,male,0,3,0,17.0,"Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...","Several days over the last 2 weeks, I was both...","Several days over the last 2 weeks, I was both...","Several days over the last 2 weeks, I was both...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...",3,This patient has not been diagnosed with depre...,Total PHQ8 score is 3.
2,302,how are you doing today. i'm fine how about yo...,male,0,4,0,28.0,"Several days over the last 2 weeks, I was both...","Over the last 2 weeks, I was not at all bother...","Nearly every day over the last 2 weeks, I was ...","Several days over the last 2 weeks, I was both...","More than half the days over the last 2 weeks,...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...",4,This patient has not been diagnosed with depre...,Total PHQ8 score is 4.
3,303,how are you doing today. okay how 'bout yourse...,female,0,0,0,17.0,"Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...","Several days over the last 2 weeks, I was both...","Several days over the last 2 weeks, I was both...","More than half the days over the last 2 weeks,...","More than half the days over the last 2 weeks,...","Several days over the last 2 weeks, I was both...","Over the last 2 weeks, I was not at all bother...",0,This patient has not been diagnosed with depre...,Total PHQ8 score is 0.
4,304,how are you doing today. i'm doing good um. wh...,female,0,6,0,20.0,"Several days over the last 2 weeks, I was both...","Nearly every day over the last 2 weeks, I was ...","More than half the days over the last 2 weeks,...","Nearly every day over the last 2 weeks, I was ...","Several days over the last 2 weeks, I was both...","More than half the days over the last 2 weeks,...","Over the last 2 weeks, I was not at all bother...","Several days over the last 2 weeks, I was both...",6,This patient has not been diagnosed with depre...,Total PHQ8 score is 6.


In [3]:
phq_binary_distribution = merged_data['PHQ_Binary'].value_counts()
phq_binary_distribution 

PHQ_Binary
0    144
1     42
Name: count, dtype: int64

In [4]:
# Initialize conversation history with the detailed system message
conversation_history = [
    {"role": "system", "content": """
You are an intelligent data generation assistant tasked with creating synthetic data for a patient questionnaire. You ensure that all words generated are properly spaced and punctuated
"""

    }
]


In [6]:
transcript_prompt_template = """
You are an intelligent data generation assistant tasked with creating synthetic data for conversation between the patient and the interviewer. The provided input is a single row from a dataframe containing patient responses. Your goal is to generate a new synthetic row that mimics the sentiment and structure of the input data.

Here is the input information:
1. Conversation: {Conversation}


Instructions:
1. Use the input conversation as reference for its structure, but create a different conversation with a different storyline, background, and locations that matches a PHQ8 score of {PHQ8_Score}. Adjust the intensity of the sentiment to align with the PHQ8 score categories: 0-4 (No or minimal depression), 5-9 (Mild depression), 10-14 (Moderate depression), 15-19 (Moderately severe depression), 20-24 (Severe depression).
2. Use the same number of columns and ensure the data is consistent with the original schema.
3. Ensure the generated data for certain categories adheres to the following rule:

- Conversation: Ensure the synthetic conversation data is grammatical and readable. 
- PHQ8_Score: Total PHQ8 score is a number from 0-24

Output the synthetic row in a compact JSON in a single-line without whitespaces for the 3 properties listed above:

1. As an example for Conversation you should have: "Conversation": "synthetic value"

Important: 
1. Ensure the generated data follows the same structural format and sentiment as the input row provided.
"""

In [7]:
# Pricing details
PROMPT_COST_PER_1000_TOKENS = 0.005
COMPLETION_COST_PER_1000_TOKENS = 0.015

# Initialize the AzureOpenAI client with the specified endpoint and API key
client = AzureOpenAI(
    azure_endpoint="https://gptshuhaotest.openai.azure.com/",  # your Azure endpoint
    api_key="d0a5a4feefc34f71b685fc394d033b2c",  # your API key
    api_version="2024-02-01"
)

MODEL = "gpt-4o"  # change model here

# # Initialize conversation history with a system message
# conversation_history = [
#     {"role": "system", "content": "You are an expert on Animals"}
# ]

# keep track of conversation
def add_message(role, content):
    conversation_history.append({"role": role, "content": content})
    
# clear conversation  
def clear_conversation():
    global conversation_history
    conversation_history = [
        {"role": "system", "content": "You are a professional psychologist who is very compassionate and empathetic"}
    ]


# Retry the function in case of an exception with an exponential backoff strategy.
# wait_random_exponential: Waits for a random amount of time, exponentially increasing between attempts.
# min: Minimum wait time is 1 second.
# max: Maximum wait time is 20 seconds.
# stop_after_attempt: Stops retrying after 10 attempts.

# @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
def get_response(prompt):
    # Add the user's message to the conversation history.
    add_message("user", prompt)
    
    # Create a chat completion request to the OpenAI API.
    response = client.chat.completions.create(
        model=MODEL,     
        # The conversation history, including system, user, and assistant messages.
        messages=conversation_history, 
        temperature=1,  # Controls the randomness of the output. 
        max_tokens=2000,  # The maximum number of tokens to generate in the response.
        top_p=1,  # Controls the diversity via nucleus sampling; 1 means use all tokens.
        frequency_penalty=2,  # Penalizes new tokens based on their frequency in the text so far.
        presence_penalty=2,  # Penalizes new tokens based on whether they appear in the text so far.
        n=1,  # The number of completions to generate. Here, we are generating only one.
        response_format={ "type": "json_object" },
    )

    # Add the assistant's message to the conversation history.
    assistant_message = response.choices[0].message.content
    add_message("assistant", assistant_message)
    
    
    # Save usage and compute cost
    usage = response.usage
    computeCost(usage)

    # Return the assistant's message.
    return assistant_message

In [8]:
import json
import pandas as pd
import random
# Initialize summary list for storing JSON data
Trancript = []

for index, row in merged_data.iterrows():
    # Determine the number of times to prompt based on PHQ_Binary value
    # prompt_multiplier = 8 if row['PHQ_Binary'] == 1 else 1
    prompt_multiplier = 3 

    for _ in range(prompt_multiplier):
        attempt_count = 0
        success = False
        PHQ8_generated = random.randint(0, 24)
        while attempt_count < 10 and not success:
            attempt_count += 1
            # Format the prompt with the current row's conversation
            prompt = transcript_prompt_template.format(
                Conversation=row['Conversation'],
                PHQ8_Score= PHQ8_generated,
            )
            try:
                # Get synthetic data response
                response = get_response(prompt)
                print(f"Participant ID {row['ParticipantID']}")
                print(f"Attempt {attempt_count}: {response}")  # More contextual printing

                # Try to parse the response as JSON
                json_data = json.loads(response)

                # Check if the required keys are present in the JSON data
                if 'PHQ8_Score' in json_data and 'Conversation' in json_data:
                    json_data['ParticipantID'] = row['ParticipantID']  # Add ParticipantID to JSON data
                    Trancript.append(json_data)
                    success = True
                else:
                    print("Error: Required keys are missing from the JSON data. Retrying...")
                    clear_conversation()
                    continue  # Continue to retry since the JSON does not have the required keys

            except json.JSONDecodeError:
                # If JSON parsing fails, retry fetching the response
                print(f"Error: Response not loadable as JSON (Attempt {attempt_count}). Retrying...")
                clear_conversation()  # Reset conversation if needed for next attempt

            except Exception as e:
                # Handle any other exceptions, including BadRequestError
                print(f"Error encountered: {e}. Skipping this prompt.")
                break  # Exit the while loop and skip to the next row

        # Clear conversation context if necessary, post all attempts
        clear_conversation()

# Convert the list of JSON objects to a DataFrame
Trancript_df = pd.DataFrame(Trancript)


Trancript_df.to_csv('syntheticTranscriptWithEllie.csv', index=False)

Total cost for this conversation: $0.01378
Participant ID 300
Attempt 1: {
"Conversation":"how are you today. not bad, a bit stressed though. where did you grow up. chicago illinois. what brought you to new york city then? i moved here for work opportunities mainly uh more jobs in my field yeah um and some friends were already living here so it made the transition easier.um how do ou enjoy your time in nyc it's definitely different from home but exciting I feel overwhelmed sometimes because it's such a huge change what's something unique about NYC well,the energy is electric really you're always busy weather isn't as great can be pretty stressful getting around mhm agreed that's life.what.did.yo.stud.at.college psychology have you've been workinginthatfield yeh i'm looking afteramily.job tha'ts flexible.whatsderealmtar.uttermomand.mentalhealthawareness.i'vealwayswntdfodothat.wow.do.youkingtow.travelman.typicalyNoI prefer staying closehto.hme familiar surroundings make.it.easier.to.mana

KeyboardInterrupt: 

In [43]:
# Trancript_df.to_csv('syntheticTranscript.csv', index=False)

In [42]:
Trancript_df.shape

(524, 6)